<a href="https://colab.research.google.com/github/krishnamani77/TF2_Notebooks/blob/master/Sentiment-analysis-IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import keras
import keras.datasets
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense, Dropout, LSTM
import numpy as np

In [19]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [21]:
# IMDB Sentiment analysis
num_words_max = 500
max_features =  50000
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)

In [22]:
word_index = keras.datasets.imdb.get_word_index()
reverse_index = {}
for k, v in word_index.items():
  reverse_index[v+3] = k

In [23]:
for id, token in enumerate(("<pad>", "<sos>", "<unk")):
  reverse_index[id] = token

In [24]:
def get_text(idx_array):
  return ' '.join([reverse_index[idx] for idx in idx_array])

In [25]:
print(get_text(X_train[0]))

<sos> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and shoul

In [26]:
X_train = sequence.pad_sequences(X_train, maxlen=num_words_max)
X_test = sequence.pad_sequences(X_test, maxlen=num_words_max)

In [27]:
def get_rnn_model(vocab_size, embed_size, input_shape):
  embedding = Embedding(vocab_size, embed_size, input_shape=(num_words_max, ))
  hidden1 = GRU(128, return_sequences = True)
  hidden2 = GRU(128)
  dense = Dense(1, activation='sigmoid')
  model = Sequential([embedding, hidden1, hidden2, dense])
   
  return model

In [28]:
def get_simple_model(input_shape):
  model = Sequential()
  # Input - Layer
  model.add(Dense(50, activation = "relu", input_shape=input_shape))
  # Hidden - Layers
  model.add(Dropout(0.3, noise_shape=None, seed=None))
  model.add(Dense(50, activation = "relu"))
  model.add(Dropout(0.2, noise_shape=None, seed=None))
  model.add(Dense(50, activation = "relu"))
  # Output- Layer
  model.add(Dense(1, activation = "sigmoid"))
  return model

In [29]:
#model = get_simple_model(input_shape=(num_words_max, ))
embed_size = 128
model = get_rnn_model(max_features, embed_size, (num_words_max, ))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=128, epochs=5)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 128)          6400000   
_________________________________________________________________
gru_3 (GRU)                  (None, 500, 128)          98688     
_________________________________________________________________
gru_4 (GRU)                  (None, 128)               98688     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 6,597,505
Trainable params: 6,597,505
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 439s 18ms/step - loss: 0.4749 - accuracy: 0.7651 - val_loss: 0.3764 - val_accuracy: 0.8362
Epoch 2/5
25000/25000 [==============================] - 442s 18ms/step - loss: 0.2373 - accuracy: 0.9090 - val_loss: 0.3590 - val_accuracy: 0.8468
Epoch 3/5
25000/25000 [==============================] - 440s 18ms/step - loss: 0.1481 - accuracy: 0.9473 - val_loss: 0.3790 - val_accuracy: 0.8581
Epoch 4/5
25000/25000 [==============================] - 444s 18ms/step - loss: 0.0701 - accuracy: 0.9772 - val_loss: 0.4249 - val_accuracy: 0.8454
Epoch 5/5
25000/25000 [==============================] - 428s 17ms/step - loss: 0.0418 - accuracy: 0.9870 - val_loss: 0.5884 - val_accuracy: 0.8536


In [1]:
#for idx_array,actual in zip(X_test[:5], y_test[:5]):
  #print(get_text(idx_array))
#  print('The predicted sentiment is {} and the actual sentiment is {} '.format(model.predict(vectorize(idx_array)), actual))

In [30]:
print(np.mean(history.history["val_accuracy"]))

0.8480400085449219
